In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

from datetime import datetime
import time
import re

import sqlite3

### 1. Web Scraping

In [7]:
url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=230&sid1=105&date=20231226&page=" + str(1)
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

urls_list=[]
for urls in source.find_all('a', {'class':'nclicks(itn.2ndcont)'}):
    if urls.get_text() != '\n\n':
        urls_list.append(urls.attrs['href'])
url=urls_list[0]

headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
web_news = requests.get(url, headers=headers).content
source_news = BeautifulSoup(web_news, 'html.parser')

date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()
date = date.replace(" ","")
date1 = date[:11] # 날짜까지 파싱
date2 = date[13:] # 시간부분 파싱
date3 = (lambda x : 'am' if x == '오전' else 'pm')(date[11:13])
date4 = date1 + date2 + date3
article_date = pd.Timestamp(date4)
                    
article_cat = source_news.find('em', {'class':'media_end_categorize_item'}).get_text()

article_press = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
                    
article_title = source_news.find('h2', {'class':'media_end_head_headline'}).get_text()
print('\nProcessing article : {}'.format(article_title))

article_document = source_news.find('article', {'id':'dic_area'}).get_text()
article_document = article_document.replace("\n", "")
article_document = article_document.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
article_document = article_document.replace("동영상 뉴스       ", "")
article_document = article_document.replace("동영상 뉴스", "")
article_document = article_document.strip()

article_link = url

print(article_date)
print(article_cat)
print(article_press)
print(article_title)



Processing article : 지금 서 있는 길이 최선일 가능성이 커요[오은영의 부모마음 아이마음]
2023-12-26 23:28:00
오피니언
동아일보
지금 서 있는 길이 최선일 가능성이 커요[오은영의 부모마음 아이마음]


In [4]:
# 기사 스크레이핑해서 result_{시간}.csv로 저장하기 
def article_scraping():
    
    date = [] # pd.timestamp 로 변환 필요
    category = [] # em class media_end_categorize_item
    press = [] # em class media_end_linked_more_point
    title = [] # h2 class media_end_head_headline 
    document = [] # article id dic_area
    link = [] 
    
    dates=['20231226', '20231227', '20231228']
    max_page = [23, 25, 24]
    
    for i, day in enumerate(dates): # 날짜별로 반복
        print('\n일시: {}'.format(day))
        
        current_page = 1
        while current_page <= max_page[i]: #페이지별로 반복
            print('\n{}번째 페이지부터 크롤링을 시작합니다.'.format(current_page))
            
            # html 소스 -> source
            url = "https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=230&sid1=105&date=" + day + "&page=" + str(current_page)
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')
    
            # 소스 페이지의 각각의 뉴스 링크 -> urls_list
            urls_list=[]
            for urls in source.find_all('a', {'class':'nclicks(itn.2ndcont)'}):
                if urls.get_text() != '\n\n':
                    urls_list.append(urls.attrs['href'])
            
            # 페이지의 링크들을 들어가 기사 scraping
            for url in urls_list:
                try:
                    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}
                    web_news = requests.get(url, headers=headers).content
                    source_news = BeautifulSoup(web_news, 'html.parser')
    
                    d = source_news.find('span', {'class':'media_end_head_info_datestamp_time'}).get_text()
                    d = d.replace(" ","")
                    date1 = d[:11] # 날짜까지 파싱
                    date2 = d[13:] # 시간부분 파싱
                    date3 = (lambda x : 'am' if x == '오전' else 'pm')(date[11:13])
                    date4 = date1 + date2 + date3
                    article_date = pd.Timestamp(date4)
                    article_cat = source_news.find('em', {'class':'media_end_categorize_item'}).get_text()
    
                    article_press = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
                    
                    article_title = source_news.find('h2', {'class':'media_end_head_headline'}).get_text()
                    print('\nProcessing article : {}'.format(article_title))
    
                    article_document = source_news.find('article', {'id':'dic_area'}).get_text()
                    article_document = article_document.replace("\n", "")
                    article_document = article_document.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                    article_document = article_document.replace("동영상 뉴스       ", "")
                    article_document = article_document.replace("동영상 뉴스", "")
                    article_document = article_document.strip()
    
                    article_link = url
                    
                    date.append(article_date)
                    category.append(article_cat)
                    press.append(article_press)
                    title.append(article_title)
                    document.append(article_document)
                    link.append(article_link)
                except:
                    print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
            
            time.sleep(5)
            current_page += 1
    
    article_df = pd.DataFrame({'date':date, 
                               'category':category, 
                               'press':press, 
                               'title':title, 
                               'document':document,
                               'link':link})

    article_df.to_csv('result_{}.csv'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
    
    print('\n크롤링이 성공적으로 완료되었습니다!')
    print('\n크롤링 결과를 다음 파일에 저장하였습니다 : {}'.format(datetime.now().strftime('%y%m%d_%H%M')))

In [5]:
article_scraping()


일시: 20231226

1번째 페이지부터 크롤링을 시작합니다.

Processing article : 지금 서 있는 길이 최선일 가능성이 커요[오은영의 부모마음 아이마음]

Processing article : 세종대 공공정책대학원, 2024학년도 전기 3차 신·편입생 모집

Processing article : 중국 당국, 고강도 게임 규제 발표… 산업계 ‘빨간불’

Processing article : “이대로 나오면 대박?” 뜻밖에 소식…삼성 모험 ‘오렌지빛’ 등장

Processing article : 미래 감염병 대비 ‘백신 개발’ 활기

Processing article : 미래 감염병 대비 ‘백신 개발’ 활기

Processing article : SF 영화가 현실로…애플 '비전프로' 1월 출시될 듯

Processing article : 사천시, 우주항공 클러스터 조성 특성화재단 설립

Processing article : 사천시, 우주항공 클러스터 조성 특성화재단 설립

Processing article : 방심위원장 “민원인 정보 유출은 범죄”…노조 “위원장 사퇴해야”

Processing article : 방심위원장 “민원인 정보 유출은 범죄”…노조 “위원장 사퇴해야”

Processing article : '특허 침해' 애플워치 미국 판매 금지…美정부, ITC 결정 확정

Processing article : 셀트리온, 오리지널 완벽 대체 ‘유플라이마’ 상호교환성 확보 추진… 관련 임상 결과 공개

Processing article : 과기정통부 “분유 제조기·전기 인덕션 등 11종 전자파 안전”

Processing article : 2023년 모바일 최다 검색 키워드는 ‘날씨’

Processing article : 구글, 직원 조직개편 예고...1년 만에 대형 구조조정 나서나

Processing article : [여기는 진주] 사천시, 우주항공 클러스터 조성 특성화재단 설립 외

Processing article : [여기는 

### 2. Database로 저장

In [6]:
dbpath = "maindb.db" # or "maindb.sqlite"

conn = sqlite3.connect(dbpath)
cur = conn.cursor()

article_df = pd.read_csv('result_231229_2229.csv')

article_df.to_sql('article', conn, index=False)

1449

In [7]:
# db 확인용 셀
script="""
SELECT *
FROM article
WHERE press="뉴시스"
"""

cur.execute(script)

df = pd.read_sql_query(script, conn)
df.head()

,date,category,press,title,document,link
0,2023-12-26 18:01:00,IT,뉴시스,"포스페이스랩, 광주경제자유구역 스타트업 발전 돕는다","포스페이스랩-광주경제자유구역청-잇그린, 혁신 스타트업 밋업 진행광주 스타트업 생태계...",https://n.news.naver.com/mnews/article/003/001...
1,2023-12-26 17:43:00,IT,뉴시스,"위메이드, 글로벌 개인정보보호 인증 APEC CBPR 취득",회원국과 사업 제휴 시 개인정보 이전 가능글로벌 시장서 게임·블록체인 사업을 효율적...,https://n.news.naver.com/mnews/article/003/001...
2,2023-12-26 17:40:00,IT,뉴시스,NXC 지분 4.7조원 인수자 한국엔 없나…결국 中·사우디?,"경영권 없는 2대 주주라 2차서도 유찰 가능성 존재거론되는 후보자는 중국 텐센트, ...",https://n.news.naver.com/mnews/article/003/001...
3,2023-12-26 17:27:00,IT,뉴시스,"이병석 전 국회부의장, 신임 한국IPTV방송협회장 내정",내년 1월 초 이사회 및 총회서 선임 확정2월 1일부터 공식 활동…임기 2년【포항=...,https://n.news.naver.com/mnews/article/003/001...
4,2023-12-26 17:26:00,IT,뉴시스,"450만원짜리 애플 비전 프로, 내년 1월 출시될 듯",애플 전문가 궈밍치 전망…1월 첫째주 대량 출하 시작[쿠퍼티노=AP/뉴시스] 애플이...,https://n.news.naver.com/mnews/article/003/001...
